In [35]:
import urllib
import numpy as np
import os
import pandas as pd
import datetime
import astropy.table
import astropy.units as u
from astropy.time import Time

def ATNFpulsarlist(save=False):
        """
        Contact the ATNF pulsar catalogue, returning an array of data.
        Each row corresponding to one pulsar, with columns in the format:
        """

        tnow = datetime.datetime.today()
        try:
            tdata = datetime.datetime.fromtimestamp(
                    os.path.getmtime('pulsar_data.txt')
                    )
        except:
            tdata = datetime.datetime(1977,1,1)
        dt = tnow - tdata
        if dt.days > 20:

            try:
                #URL to get |NAME|PSRJ|RAJ|DECJ|POSEPOCH|F0|F1|F2|PEPOCH|DM|W50|W10|S400|S1400|SPINDX|PSRTYPE|NGLT|
                url2 = 'http://www.atnf.csiro.au/research/pulsar/psrcat/proc_form.php?version=1.54&Name=Name&JName=JName&RaJ=RaJ&DecJ=DecJ&PosEpoch=PosEpoch&F0=F0&F1=F1&F2=F2&PEpoch=PEpoch&DM=DM&DIST=DIST&W50=W50&W10=W10&S400=S400&S1400=S1400&SPINDX=SPINDX&Type=Type&NGlt=NGlt&startUserDefined=true&c1_val=&c2_val=&c3_val=&c4_val=&sort_attr=jname&sort_order=asc&condition=&pulsar_names=&ephemeris=short&coords_unit=raj/decj&radius=&coords_1=&coords_2=&style=Short+without+errors&no_value=*&fsize=3&x_axis=&x_scale=linear&y_axis=&y_scale=linear&state=query&table_bottom.x=40&table_bottom.y=24'
                Hurl2='#NAME|PSRJ|RAJ|DECJ|POSEPOCH|F0|F1|F2|PEPOCH|DM|W50|W10|S400|S1400|SPINDX|PSRTYPE|NGLT\n'
                sock = urllib.urlopen(url2)
                data = sock.read()
                sock.close()


                data = data.split('<pre>')[1]
                data = data.split('</pre>')[0]
                data = data.splitlines()[5:-1]

                header = "Name|PSRJ|RAJ|DECJ|POSEPOCH|F0|F1|F2|PEPOCH|DM|DIST|W50|W10|S400|S1400|SPINDX|PSRTYPE|NGLT".split('|')

                df = pd.DataFrame(columns=header)

                for b in data:
                    b = b.split()
                    try:
                        df = df.append(pd.Series(b[1:], index=header, name=b[0]))
                    except:
                        pass
                del(df[''])

                

            except:
                pass
        
            df.to_csv('pulsar_data.txt', header=header)
        else:
            df = pd.read_csv('pulsar_data.txt', na_values='*')
        
        a = astropy.table.Table.from_pandas(df)
        
        a['RAJ'].unit = u.hourangle
        a['DECJ'].unit = u.deg
        a['F0'].unit = u.hertz
        a['F1'].unit = u.hertz / u.second
        a['F2'].unit = u.hertz / u.second**2
        a['DIST'].unit = u.kiloparsec
        a['W50'].unit = u.millisecond
        a['W10'].unit = u.millisecond
        a['DM'].unit = u.centimeter**(-3) * u.parsec
        #a['EDOT'].unit = u.erg / u.second
        a.add_index('Unnamed: 0')
        a.rename_column('Unnamed: 0', '#')
        a['POS'] = SkyCoord(a['RAJ'], a['DECJ'], unit=(a['RAJ'].unit, a['DECJ'].unit))
        return a

In [36]:
a = ATNFpulsarlist()

In [37]:
a

#,Name,PSRJ,RAJ,DECJ,POSEPOCH,F0,F1,F2,PEPOCH,DM,DIST,W50,W10,S400,S1400,SPINDX,PSRTYPE,NGLT,POS
,,,hourangle,deg,,Hz,Hz / s,Hz / s2,,pc / cm3,kpc,ms,ms,,,,,,"deg,deg"
int64,str12,str12,str10,str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str25,float64,object
1,J0006+1834,J0006+1834,00:06:04.8,+18:34:59,49079.5,1.441446,-4.357e-15,--,49079.5,12.0,0.7,82.0,195.0,0.2,--,--,--,--,"1.52,18.5830555556"
2,J0007+7303,J0007+7303,00:07:01.7,+73:03:07.4,54952.0,3.165827,-3.612e-12,4.1e-23,54952.0,--,1.4,--,--,--,--,--,NRAD,1.0,"1.75708333333,73.0520555556"
3,B0011+47,J0014+4746,00:14:17.7,+47:46:33.4,49664.0,0.805997,-3.667e-16,7.3e-28,49664.0,30.85,1.82,88.7,142.5,14.0,3.0,-1.3,--,--,"3.57375,47.7759444444"
4,J0023+0923,J0023+0923,00:23:16.8,+09:23:23.8,56179.0,327.868852,--,--,--,14.3,1.0,--,--,2.0,--,--,HE,--,"5.82,9.38994444444"
5,B0021-72C,J0024-7204C,00:23:50.3,-72:04:31.4,51600.0,173.708219,1.504e-15,--,51600.0,24.6,4.0,0.8,--,1.53,0.6,--,--,--,"5.95958333333,-72.0753888889"
6,B0021-72D,J0024-7204D,00:24:13.8,-72:04:43.8,51600.0,186.65167,1.195e-16,--,51600.0,24.73,4.0,0.6,--,0.95,--,--,--,--,"6.0575,-72.0788333333"
7,B0021-72E,J0024-7204E,00:24:11.1,-72:05:20.1,51600.0,282.779107,-7.877e-15,--,51600.0,24.23,4.0,0.9,--,--,--,--,--,--,"6.04625,-72.0889166667"
8,B0021-72F,J0024-7204F,00:24:03.8,-72:04:42.8,51600.0,381.158664,-9.371e-15,--,51600.0,24.38,4.0,0.5,--,--,--,--,--,--,"6.01583333333,-72.0785555556"


In [ ]:
from astropy.coordinates import SkyCoord, ICRS

In [24]:
import matplotlib.pyplot as plt

In [25]:
a['POS'][0].separation(a['POS'][1])

<Angle 54.46916653982224 deg>

In [26]:
a

#,Name,PSRJ,RAJ,DECJ,POSEPOCH,F0,F1,F2,PEPOCH,DM,DIST,W50,W10,S400,S1400,SPINDX,PSRTYPE,NGLT,POS
,,,hourangle,deg,,Hz,Hz / s,Hz / s2,mjupiterMass,pc / cm3,kpc,ms,ms,,,,,,"deg,deg"
int64,str12,str12,str10,str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str25,float64,object
1,J0006+1834,J0006+1834,00:06:04.8,+18:34:59,49079.5,1.441446,-4.357e-15,--,49079.5,12.0,0.7,82.0,195.0,0.2,--,--,--,--,"1.52,18.5830555556"
2,J0007+7303,J0007+7303,00:07:01.7,+73:03:07.4,54952.0,3.165827,-3.612e-12,4.1e-23,54952.0,--,1.4,--,--,--,--,--,NRAD,1.0,"1.75708333333,73.0520555556"
3,B0011+47,J0014+4746,00:14:17.7,+47:46:33.4,49664.0,0.805997,-3.667e-16,7.3e-28,49664.0,30.85,1.82,88.7,142.5,14.0,3.0,-1.3,--,--,"3.57375,47.7759444444"
4,J0023+0923,J0023+0923,00:23:16.8,+09:23:23.8,56179.0,327.868852,--,--,--,14.3,1.0,--,--,2.0,--,--,HE,--,"5.82,9.38994444444"
5,B0021-72C,J0024-7204C,00:23:50.3,-72:04:31.4,51600.0,173.708219,1.504e-15,--,51600.0,24.6,4.0,0.8,--,1.53,0.6,--,--,--,"5.95958333333,-72.0753888889"
6,B0021-72D,J0024-7204D,00:24:13.8,-72:04:43.8,51600.0,186.65167,1.195e-16,--,51600.0,24.73,4.0,0.6,--,0.95,--,--,--,--,"6.0575,-72.0788333333"
7,B0021-72E,J0024-7204E,00:24:11.1,-72:05:20.1,51600.0,282.779107,-7.877e-15,--,51600.0,24.23,4.0,0.9,--,--,--,--,--,--,"6.04625,-72.0889166667"
8,B0021-72F,J0024-7204F,00:24:03.8,-72:04:42.8,51600.0,381.158664,-9.371e-15,--,51600.0,24.38,4.0,0.5,--,--,--,--,--,--,"6.01583333333,-72.0785555556"
